# Calibration of GEOtop using Nevergrad and GEOtoPy

In [ ]:
# Default

model_path = '../data/Matsch B2/inputs'
timeout = 240
observations_path = '../data/Matsch B2/observations/obs.csv'
parameters_path = '../data/Calibration parameters/default_parameters_table.csv'
spinup = {'years': 1}
skip_months = None
default_parameters = None
algorithm = 'NGO'
budget = 8
popsize = 2
address = 'local'
maximum_jobs = 64

In [ ]:
from dateutil.relativedelta import relativedelta

import pandas as pd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster as Cluster

from mhpc_project.utils import date_parser
from mhpc_project.parameters import VarSoilParameters
from mhpc_project.models import VarSoilFullModel
from mhpc_project.calibration import Calibration

In [ ]:
parameters = VarSoilParameters(parameters_path, default_parameters)
model = VarSoilFullModel(model_path, timeout=timeout)
observations = pd.read_csv(observations_path,
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)

if spinup:
    start_date = date_parser(model.settings['InitDateDDMMYYYYhhmm'])
    start_date = start_date + relativedelta(**spinup)
    selection = observations.index.map(lambda x: x >= start_date)
    observations = observations[selection]

if skip_months:
    selection = observations.index.map(lambda x: x.month not in skip_months)
    observations = observations[selection]

calibration = Calibration(model, parameters, observations)

In [ ]:
if address:
    if address == 'local':
        client = Client()
    else:
        client = Client(address=address)
else:
    cluster = Cluster()
    cluster.adapt(maximum_jobs=maximum_jobs)

    client = Client(cluster)

In [ ]:
initial_guess = calibration.parameters.instrumentation
calibration.comparison_plots(initial_guess)

In [ ]:
%%time
calibrated_output, recommendation = calibration(algorithm, budget, popsize, client, show_progress=True)

In [ ]:
calibration.comparison_plots(recommendation)

In [ ]:
calibration.parameters_table(recommendation)